In [1]:
import sys
import pandas as pd
import sqlite3
import os
import glob
import copy
import datetime
import numpy as np
from struct import *

In [11]:
def Savetosqlite3(df,DB_Name = 'F:/sqlite3/tdx.db',sql='insert into test(ts_code,trade_date,close,open,high,low,pct_chg,vol,amount,name) values(?,?,?,?,?,?,?,?,?,?,?)'):    
    '''
    将dataframe数据存到sqlite3中
    '''
    conn = sqlite3.connect(DB_Name)  
    cursor = conn.cursor()    # 向sqlite3中表插入数据的SQL语句
    print('连接数据库%s成功' % (DB_Name))
    #print(df)
    refresh_date = datetime.date.today() #今天日期
    df['refresh_date'] = refresh_date
    
    '''
    try
        df.drop(columns='结算价',inplace=True)
        print('已删除结算价格')
    except Unexpected as e:
        print('无结算价')
    '''
    sqldata = [tuple(xi) for xi in df.values]
    print('转换为元组')
    try:
        cursor.executemany(sql,sqldata)    #每日的数据添加一次，不再汇总后添加
        print('连不上')
    except Unexpected as e:              
        conn.rollback() # 回滚
        print('插入数据到表失败')               
    print ( 'has been saved in SQLite.TDX的交易所中!' )
    # 批量插入之后再执行事务提交
    conn.commit()  
    # 关闭游标
    cursor.close()
    # 断开数据库连接
    conn.close()

### 读取通达信导出的TXT文件，存入sqlite3中，推荐使用

def save_TXTdata(file_address,DB_Name,sql):
    '''
    读取通达信导出的TXT文件，存入sqlite3中。通达信导出的数据中含有代码和名字，取国际指数和其他经济数据比较方便.tdx导出txt文本时，
    使用TAB（）格式，使用时间格式为YYYYMMDD,与tushare网站数据一致。建议使用TXT文本数据，方便数据库统一格式。
    '''
    os.chdir(file_address)
    files_list = glob.glob('*.txt')
    print(len(files_list))
    address = os.getcwd()#返回当前目录

    for files in files_list:
        try:
            #header =0,读入文件，txt文件中的第一行保存，然后代码和名称读出为nameN和nameM，skipfooter=1,删除最后一行的通达信数据源，delim_whitespace=True分隔符为tab
            data = pd.read_table(file_address+files,delim_whitespace=True,encoding='gb2312',skipfooter=1,engine='python',parse_dates=True,infer_datetime_format=True) 
            nameN = copy.deepcopy(data.columns[0])
            nameM = copy.deepcopy(data.columns[1])
            '''不要st股票,#name1 = re.sub(r'\/' , 'x',s1)
            if ('*st' in nameM):
                print(files_list[i] +' is *ST股票，删除中...')
                continue;
            '''
            #header =1，再次读入文件，包含txt文本中的第一行，并把刚刚读出的代码和名称写入
            data = pd.read_table(file_address+files,delim_whitespace=True,header =1,encoding='gb2312',skipfooter=1,engine='python',parse_dates=True,infer_datetime_format=True) 
            #data = pd.read_table(file_address +'\\'+ files_list[i],sep=',',encoding='gb2312',header =1,parse_dates=True,infer_datetime_format=True) 
            data.insert(0,'code',nameN)
            data.insert(8,'name',nameM)
            
            #data.rename(index=str, columns={"开盘": "open", "收盘": "close", "成交量": "vol", "日期": "date", "最低": "low", "最高": "high", "成交额": "amount"},inplace=True)
            #data.to_csv('D'+nameN+'.csv',sep=',',encoding='gb2312')
            data['pct_chg'] = round(data['收盘'].diff()/data['收盘'] * 100,4)#涨跌娶4位有效数字
            data.iloc[:,6]   = data.iloc[:,6]/100.0                             #成交量为手
            data.iloc[:,7]   = data.iloc[:,7]/10000.0                           #成交额或者持仓量为万
            #print(data.head())
            Savetosqlite3(data,DB_Name,sql)  #保存到数据库
            print( nameM  +' has been saved!' )
        except UnicodeDecodeError:
            print(files +'gb2312 codec can not decode')
        except:
            print(files +" has Unexpected error:")     



def read_SQlite3(sql,addr="F:/sqlite3/tdx.db"):
    '''
    sql中需要写明"select * from index_daily where trade_date Between '20200501' AND '20200724'" 读取的table和条件 
    '''
    con = sqlite3.connect(addr)
    cur = con.cursor()
    sql = "select * from index_daily where trade_date Between '20200501' AND '20200724'"
    cur.execute(sql)
    df = cur.fetchall()
    cur.close()
    con.close()
    
    return df#返回数组格式

In [12]:

def save_funddata(file_address,DB_Name,sql,all_fund):
    '''
    读取通达信导出的TXT文件，存入sqlite3中。通达信导出的数据中含有代码和名字，取国际指数和其他经济数据比较方便.tdx导出txt文本时，
    使用TAB（）格式，使用时间格式为YYYYMMDD,与tushare网站数据一致。建议使用TXT文本数据，方便数据库统一格式。
    '''

    os.chdir(file_address)
    files_list = glob.glob('*.txt')
    print(len(files_list))
    address = os.getcwd()#返回当前目录

    for files in files_list:
        try:

            #header =0,读入文件，txt文件中的第一行保存，然后代码和名称读出为nameN和nameM，skipfooter=1,删除最后一行的通达信数据源，delim_whitespace=True分隔符为tab
            data = pd.read_table(file_address+files,delim_whitespace=True,header =0,skipfooter=1,engine='python') #delim_whitespace=True,识别tab 空格
            
            nameN = str(copy.deepcopy(data.columns[0]))
            nameM = str(copy.deepcopy(data.columns[1]))            
            #header =1，再次读入文件，包含txt文本中的第一行，并把刚刚读出的代码和名称写入
            data = pd.read_table(file_address+files,delim_whitespace=True,header =1,skipfooter=1,engine='python') 

            data.insert(0,'code',nameN)
            data.insert(1,'name',nameM)
            data.drop(columns=['开盘','最高','最低','成交量','成交额'],inplace=True)
            data['pct_chg'] = round(data['收盘'].diff()/data['收盘'] * 100,4)#涨跌娶4位有效数字


            data['fund_type'] = all_fund.loc[all_fund['代码']=='000017','产品类型'].values[0]#用values[0]把格式转换为字符型，然后给列赋值
            data['style'] =  all_fund.loc[all_fund['代码']=='000017','投资风格'].values[0]
            data['state'] =  all_fund.loc[all_fund['代码']=='000017','状态'].values[0]
            data['rise_day'] =  all_fund.loc[all_fund['代码']=='000017','连涨天'].values[0]
            data['near_week'] = all_fund.loc[all_fund['代码']=='000017','近一周%'].values[0]
            data['near_month'] = all_fund.loc[all_fund['代码']=='000017','近一月%'].values[0]
            data['near_3month'] = all_fund.loc[all_fund['代码']=='000017','近三月%'].values[0]
            data['near_year'] = all_fund.loc[all_fund['代码']=='000017','近一年%'].values[0]
            data['total_amount'] = all_fund.loc[all_fund['代码']=='000017','最近份额'].values[0]
            data['company'] = all_fund.loc[all_fund['代码']=='000017','基金公司'].values[0]

            data['startdate'] = all_fund.loc[all_fund['代码']=='000017','成立日期'].values[0]
            data['buyer_num'] = all_fund.loc[all_fund['代码']=='000017','认购户数'].values[0]
            data['master'] = all_fund.loc[all_fund['代码']=='000017','投资经理'].values[0]           
            Savetosqlite3(data,DB_Name,sql)  #保存到数据库

        except UnicodeDecodeError:
            print(files +'gb2312 codec can not decode')
        except:
            print(files +" has Unexpected error:")
           

In [14]:
if __name__ == "__main__":
    #定义全局变量
    all_fund = pd.read_table('E:/20200803.txt',header =0,skipfooter=1,engine='python')#skipfooter=1,删掉最后一行

    all_fund['代码'] = all_fund['代码'].astype(str)
    all_fund['代码'] = all_fund['代码'].str.zfill(6)
    all_fund.drop(columns='Unnamed: 20',inplace= True)
    sql = 'insert into fund(ts_code,name,trade_date,new_price,pct_chg,fund_type,style,state,rise_day,near_week,near_month,near_3month,near_year,total_amount,company,startdate,buyer_num,master,refresh_date) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'    
    save_funddata('E:/sqlite3/','E:/sqlite3/tdx.db',sql,all_fund)


1307
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved i

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

...

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.

连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!
连接数据库E:/sqlite3/tdx.db成功
转换为元组
连不上
has been saved in SQLite.TDX的交易所中!


In [5]:
    hs300.drop(columns='Unnamed: 0',inplace=True)
    hs300['股票'] =  hs300['股票'].astype(str).str.zfill(6)
    symbol_dict = dict(zip(hs300.股票,hs300.股票名称))
    symbols, names = np.array(sorted(symbol_dict.items())).T
    
    quotes = []  #全局变量
    
    for symbol in symbols:
        #print('Fetching quote history for %r' % symbol, file=sys.stderr)
        temp = get_daily_data(symbol)
        quotes.append(temp)

NameError: name 'hs300' is not defined

In [ ]:
close_prices = np.vstack([q['close'] for q in quotes])
#open_prices = np.vstack([q['open'] for q in quotes])
close_change = close_prices[0][1:]-close_prices[0][0:-1]

# The daily variations of the quotes are what carry most information
variation = np.delete(close_prices,0, axis=1)
close_prices = np.delete(close_prices, -1, axis=1)
change = variation/close_prices                                  #修改为每日涨跌幅对比

# #############################################################################
# Learn a graphical structure from the correlations
#edge_model =  covariance.GraphLassoCV()
edge_model = covariance.MinCovDet()
# standardize the time series: using correlations rather than covariance
# is more efficient for structure recovery
X = change.copy().T
X /= X.std(axis=0)
edge_model.fit(X)

# #############################################################################
# Cluster using affinity propagation
#hs300.to_csv('D:/data/hs302.csv', encoding='utf-8')
symbol_dict = dict(zip(hs300.股票,hs300.股票名称))
symbols,names = np.array(sorted(symbol_dict.items())).T
_, labels = cluster.affinity_propagation(edge_model.covariance_)

n_labels = labels.max()
for i in range(n_labels+1):
    print('Cluster %i: %s' % ((i+1), ', '.join(names[labels == i])))
# #############################################################################
# Find a low-dimension embedding for visualization: find the best position of
# the nodes (the stocks) on a 2D plane

# We use a dense eigen_solver to achieve reproducibility (arpack is
# initiated with random vectors that we don't control). In addition, we
# use a large number of neighbors to capture the large-scale structure.
node_position_model = manifold.LocallyLinearEmbedding(
    n_components=2, eigen_solver='dense', n_neighbors=6)

embedding = node_position_model.fit_transform(X.T).T

# #############################################################################
# Visualization
plt.figure(1, facecolor='w', figsize=(30, 20))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis('off')

# Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02)

# Plot the nodes using the coordinates of our embedding
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
            cmap=plt.cm.nipy_spectral)

# Plot the edges
start_idx, end_idx = np.where(non_zero)
# a sequence of (*line0*, *line1*, *line2*), where::
#            linen = (x0, y0), (x1, y1), ... (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
            for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero])
lc = LineCollection(segments,
                    zorder=0, cmap=plt.cm.hot_r,
                    norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(15 * values)
ax.add_collection(lc)

# Add a label to each node. The challenge here is that we want to
# position the labels to avoid overlap with other labels
for index, (name, label, (x, y)) in enumerate(
        zip(names, labels, embedding.T)):

    dx = x - embedding[0]
    dx[index] = 1
    dy = y - embedding[1]
    dy[index] = 1
    this_dx = dx[np.argmin(np.abs(dy))]
    this_dy = dy[np.argmin(np.abs(dx))]
    if this_dx > 0:
        horizontalalignment = 'left'
        x = x + .002
    else:
        horizontalalignment = 'right'
        x = x - .002
    if this_dy > 0:
        verticalalignment = 'bottom'
        y = y + .002
    else:
        verticalalignment = 'top'
        y = y - .002
    plt.text(x, y, name, size=10,
             horizontalalignment=horizontalalignment,
             verticalalignment=verticalalignment,
             bbox=dict(facecolor='w',
                       edgecolor=plt.cm.nipy_spectral(label / float(n_labels)),
                       alpha=.6))

plt.xlim(embedding[0].min() - .15 * embedding[0].ptp(),
         embedding[0].max() + .10 * embedding[0].ptp(),)
plt.ylim(embedding[1].min() - .03 * embedding[1].ptp(),
         embedding[1].max() + .03 * embedding[1].ptp())

plt.show()